In [ ]:
pip install jupyter_dash
pip install dash_bootstrap_components
pip install dash_bootstrap_templates

In [ ]:
#Please setup your token here
px.set_mapbox_access_token()

In [ ]:
resorts = (
    pd.read_csv("resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_cannon_rank = lambda x: x.groupby("Country", as_index=False)["Snow cannons"].rank(ascending=False),
    ))


In [ ]:
# Define external Bootstrap themes to be used for dynamic theme switching.
external_themes = {
    'Bootstrap': 'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',
    'Cerulean': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/cerulean/bootstrap.min.css',
    'Cosmo': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/cosmo/bootstrap.min.css',
    'Flatly': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/flatly/bootstrap.min.css',
    'Journal': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/journal/bootstrap.min.css',
    'Lumen': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/lumen/bootstrap.min.css',
    'Minty': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/minty/bootstrap.min.css',
    'Pulse': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/pulse/bootstrap.min.css',
    'Sandstone': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/sandstone/bootstrap.min.css',
    'Simplex': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/simplex/bootstrap.min.css',
    'Spacelab': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/spacelab/bootstrap.min.css',
    'United': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/united/bootstrap.min.css',
    'Yeti': 'https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/yeti/bootstrap.min.css'
}

app = Dash(__name__)

# The layout of the app
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='theme-selector',
                options=[{'label': theme_name, 'value': theme_url} for theme_name, theme_url in external_themes.items()],
                value=external_themes['Cerulean'],
                clearable=True,
                style={'width': '200px', 'marginBottom': '20px'}
            ),
            html.Link(
                rel='stylesheet',
                href=external_themes['Cerulean'],
                id='theme-link'
            )
        ], width=3),
    ]),

    dcc.Tabs(className="dbc", children=[
        dbc.Tab(label="Resort Map", children=[
            html.H1(id="map-title", style={"text-align": "center"}),
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        html.H5("Price Range", className="card-header", style={'textAlign': 'center'}),
                        dcc.Slider(id="price-slider", min=0, max=150, step=25, value=150, className="dbc"),
                    ], style={'marginBottom': 80, 'marginTop': 60}),

                    dbc.Card([
                        html.H5("Feature Preferences", className="card-header", style={'textAlign': 'center'}),
                        dcc.Checklist(
                            id="summer-ski",
                            options=[{"label": "Summer Skiing", "value": "Yes"}],
                            value=[],
                            className="my-1"
                        ),
                        dcc.Checklist(
                            id="night-ski",
                            options=[{"label": "Night Skiing", "value": "Yes"}],
                            value=[],
                            className="my-1"
                        ),
                        dcc.Checklist(
                            id="snow-park",
                            options=[{"label": "Snow Park", "value": "Yes"}],
                            value=[],
                            className="my-1"
                        ),
                    ])
                ], width=3),
                dbc.Col(dcc.Graph(id="resort-map"), width=9)
            ])
        ]),
        dbc.Tab(label="Country Profiler", children=[
            html.H1(id="country-title", style={"text-align": "center"}),
            dbc.Row([
                dbc.Col([
                    dcc.Markdown("Select Continent:"),
                    dcc.Dropdown(
                        id="continent-dropdown",
                        options=[{"label": c, "value": c} for c in resorts["Continent"].unique()],
                        value="North America",
                        className="dbc"
                    ),
                    html.Br(),
                    dcc.Markdown("Select Country:"),
                    dcc.Dropdown(
                        id="country-dropdown",
                        options=[{"label": c, "value": c} for c in resorts["Country"].unique()],
                        value="Canada",
                        className="dbc"
                    ),
                    html.Br(),
                    dcc.Markdown("Select Metric to Plot:"),
                    dcc.Dropdown(
                        id="column-picker",
                        options=[{"label": col, "value": col} for col in resorts.select_dtypes("number").columns[3:]],
                        value="Price",
                        className="dbc"
                    ),
                ], width=3),
                dbc.Col([dcc.Graph(id="metric",
                                   hoverData={'points': [{'customdata': ['Hemsedal']}]})],
                        width=6),
                dbc.Col([
                    dcc.Markdown("#### Resort Report Card"),
                    dbc.Card(id="resort-name", style={"text-align": "center", "fontSize": 20}),
                    dbc.Row([
                        dbc.Col([dbc.Card(id="elevation"), dbc.Card(id="price")]),
                        dbc.Col([dbc.Card(id="slope"), dbc.Card(id="cannon")]),
                    ])
                ], width=3)
            ])
        ])
    ])
], style={"width": 1300})

# Callbacks

@app.callback(
    Output("country-dropdown", "options"),
    Input("continent-dropdown", "value"))
def country_select(continent):
    """
    Dynamically updates the country dropdown options based on the selected continent.

    When a continent is selected in the 'continent-dropdown' menu, this callback fetches
    and sorts the countries from the dataset that are associated with that continent.
    It then updates the 'country-dropdown' with these sorted country names, enabling user
    interaction and subsequent selections.

    Args:
        continent (str): The continent selected by the user. It is used to filter the
                         countries displayed in the dropdown.

    Returns:
        list of str: A list containing sorted country names from the dataset that
                     correspond to the selected continent.
    """
    return np.sort(resorts.query("Continent == @continent").Country.unique())

@app.callback(
    Output("country-title", "children"),
    Output("metric", "figure"),
    Input("country-dropdown", "value"),
    Input("column-picker", "value")
)
def plot_bar(country, metric):
    """
    Generates and updates a bar chart reflecting the rankings of ski resorts within a selected country based on a specified metric.

    This callback function is triggered whenever the user selects a different country or metric from the respective dropdown menus.
    It retrieves and sorts the data for the selected country by the chosen metric and updates the bar chart to display these rankings.
    If either input is missing, the update is prevented to avoid errors.

    Args:
        country (str): The country selected by the user from the dropdown, used to filter the dataset for specific country data.
        metric (str): The metric (such as Price or Elevation) selected by the user from the dropdown, used for sorting the resort data.

    Returns:
        tuple:
            str: The dynamically generated title of the bar chart, which includes the selected country and metric.
            plotly.graph_objs._figure.Figure: The updated bar chart displaying sorted data of ski resorts based on the selected metric.

    Raises:
        PreventUpdate: An exception used by Dash to prevent the callback from firing if the inputs are not valid or insufficient.

    Example:
        If the inputs are "Canada" for the country and "Price" for the metric, the function will generate a bar chart
        titled "Top Resorts in Canada by Price" that ranks resorts by their pricing in ascending order.
    """
    if not country and metric:
        raise PreventUpdate
    title = f"Top Resorts in {country} by {metric}"

    df = resorts.query("Country == @country").sort_values(metric, ascending=False)

    figure = px.bar(df, x="Resort", y=metric, custom_data=["Resort"]).update_xaxes(showticklabels=False)

    return title, figure

@app.callback(
    Output("resort-name", "children"),
    Output("elevation", "children"),
    Output("price", "children"),
    Output("slope", "children"),
    Output("cannon", "children"),
    Input("metric", "hoverData"))
def report_card(hoverData):
    """
    Updates the detailed report card for a ski resort based on user interactions with a bar chart.

    This callback function is triggered when a user hovers over a bar in the chart. It retrieves and displays detailed
    information about the selected ski resort, such as its name and rankings within its country for elevation, price,
    slopes, and snow cannons. Each of these metrics is displayed in a dedicated section of the report card.

    Args:
        hoverData (dict): Data related to the hover event on the bar chart, containing information about the
                          selected resort, such as its name and other custom data.

    Returns:
        tuple:
            - str: The name of the selected ski resort.
            - str: The rank of the resort's elevation compared to other resorts in the same country.
            - str: The rank of the resort's price compared to other resorts in the same country.
            - str: The rank of the resort's total slopes compared to other resorts in the same country.
            - str: The rank of the resort's snow cannons compared to other resorts in the same country.

    Example:
        If the user hovers over a resort named "Snowy Peaks" in the bar chart, this function will fetch and display:
            - The resort's name: "Snowy Peaks"
            - Its elevation rank: "Elevation Rank: 5"
            - Its price rank: "Price Rank: 3"
            - Its slope rank: "Slope Rank: 8"
            - Its snow cannon rank: "Cannon Rank: 2"
    """
    resort = hoverData["points"][0]["customdata"][0]
    df = resorts.query("Resort == @resort")

    elev_rank = f"Elevation Rank: {int(df['country_elevation_rank'])}"
    price_rank = f"Price Rank: {int(df['country_price_rank'])}"
    slope_rank = f"Slope Rank: {int(df['country_slope_rank'])}"
    cannon_rank = f"Cannon Rank: {int(df['country_cannon_rank'])}"

    return resort, elev_rank, price_rank, slope_rank, cannon_rank

def update_theme(selected_theme):
    return selected_theme

if __name__ == "__main__":
    app.run_server(port=2034)